**import libraries**

In [1]:
import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn import utils
from glob import glob

In [2]:
import librosa as lr

**upload dataset**

In [45]:
!gdown --id 1--C4P9SSZhYnTITWyYpEzNXZwpOKM6tW

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1--C4P9SSZhYnTITWyYpEzNXZwpOKM6tW
To: /content/dataset.zip
100% 35.0M/35.0M [00:00<00:00, 261MB/s]


In [46]:
!unzip dataset.zip

Archive:  dataset.zip
replace dataset/1/1-(1).mp3? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
labels = ['اوراق', 'ارز', 'سکه', 'بانک', 'طلا', 'نفت', 'مشتقات', 'فلزات', 'صندوق سهامی', 'صندوق درآمد ثابت', 'صندوق مختلط', 'صندوق قابل معامله']

In [ ]:
len(labels)

12

### **Preprocessing data**

### **Read dataset**

**read data.json**

In [5]:
!gdown --id 1-322-3NQ84fc2aM2Xg1_LsMf6H8STsZ5

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-322-3NQ84fc2aM2Xg1_LsMf6H8STsZ5
To: /content/data.json
100% 270M/270M [00:02<00:00, 129MB/s]


In [6]:
def load_data(data_path):
    with open(data_path, "r") as f:
        data = json.load(f)

    X = np.array(data["MFCCs"])
    y = np.array(data["labels"])
    return X, y

In [7]:
X, y = load_data('data.json')

In [8]:
X.shape

(2880, 20, 300)

In [9]:
y[0:40]

array(['اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق',
       'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق',
       'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق',
       'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق',
       'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق',
       'اوراق', 'اوراق', 'اوراق', 'اوراق', 'اوراق'], dtype='<U17')

In [10]:
df = pd.DataFrame({'signal': X.tolist(), 'category': y.tolist()})

In [11]:
df.head()

,signal,category
0,"[[-591.5642700195312, -591.5642700195312, -591...",اوراق
1,"[[-267.8851324246888, -265.87160453519965, -26...",اوراق
2,"[[-536.2584228515625, -531.6232299804688, -529...",اوراق
3,"[[-601.7645263671875, -601.7645263671875, -601...",اوراق
4,"[[-616.0204467773438, -616.0204467773438, -616...",اوراق


In [12]:
categories = pd.get_dummies(y).columns.values

In [13]:
categories

array(['ارز', 'اوراق', 'بانک', 'سکه', 'صندوق درآمد ثابت', 'صندوق سهامی',
       'صندوق قابل معامله', 'صندوق مختلط', 'طلا', 'فلزات', 'مشتقات',
       'نفت'], dtype=object)

In [14]:
y_one_hot = pd.get_dummies(y).values

In [15]:
y_one_hot

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [16]:
len(X)

2880

In [17]:
def get_train_test(X, y_one_hot):
  X_train = []
  X_test = []
  Y_train = []
  Y_test = []
  counter = 0
  for i in range(len(X)):
    if (i+1) % 40 == 0:
      x_train, x_test, y_train, y_test = train_test_split(X[counter*40:(counter*40)+40], y_one_hot[counter*40:(counter*40)+40], test_size=0.2, shuffle=True, random_state=10)
      
      X_train += x_train.tolist()
      X_test += x_test.tolist()
      Y_train += y_train.tolist()
      Y_test += y_test.tolist()  

      counter += 1
  
  return np.array(X_train), np.array(X_test), np.array(Y_train), np.array(Y_test)

In [18]:
X_train, X_test, y_train, y_test = get_train_test(X, y_one_hot)

In [19]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2304, 20, 300)
(576, 20, 300)
(2304, 12)
(576, 12)


**shuffle dataset**

In [20]:
X_train, y_train = utils.shuffle(X_train, y_train)
X_test, y_test = utils.shuffle(X_test, y_test)

In [21]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

**add channel**

In [22]:
print(X_train.shape)
print(X_test.shape)

(2304, 20, 300, 1)
(576, 20, 300, 1)


### **Build Model**

In [23]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(64, kernel_size=(2, 2), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))
model.add(tf.keras.layers.MaxPooling2D((2, 2), padding='same'))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(2, 2), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.MaxPooling2D((2, 2), padding='same'))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(2, 2), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.MaxPooling2D((2, 2), padding='same'))
model.add(tf.keras.layers.Conv2D(128, kernel_size=(2, 2), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.MaxPooling2D((2, 2), padding='same'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(categories), activation='softmax'))

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 19, 299, 64)       320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 10, 150, 64)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 149, 32)        8224      
                                                                 
 dropout (Dropout)           (None, 9, 149, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 75, 32)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 74, 32)         4

In [25]:
model.compile(
              optimizer=tf.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [26]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", min_delta=0.001, patience=5)

In [27]:
history = model.fit(
              X_train,
              y_train,
              batch_size=8,
              epochs=300,
              verbose=1,
              validation_data=(X_test, y_test),
              #callbacks=[earlystop_callback]
              )

Epoch 1/300
288/288 [==============================] - 11s 7ms/step - loss: 4.4508 - accuracy: 0.0946 - val_loss: 2.5966 - val_accuracy: 0.1042
Epoch 2/300
288/288 [==============================] - 2s 6ms/step - loss: 2.6737 - accuracy: 0.0964 - val_loss: 2.6006 - val_accuracy: 0.0851
Epoch 3/300
288/288 [==============================] - 2s 6ms/step - loss: 2.6271 - accuracy: 0.0864 - val_loss: 2.5980 - val_accuracy: 0.0851
Epoch 4/300
288/288 [==============================] - 2s 5ms/step - loss: 2.6061 - accuracy: 0.0894 - val_loss: 2.5935 - val_accuracy: 0.1024
Epoch 5/300
288/288 [==============================] - 2s 6ms/step - loss: 2.6084 - accuracy: 0.0920 - val_loss: 2.5962 - val_accuracy: 0.0868
Epoch 6/300
288/288 [==============================] - 2s 6ms/step - loss: 2.6020 - accuracy: 0.0855 - val_loss: 2.5921 - val_accuracy: 0.0955
Epoch 7/300
288/288 [==============================] - 2s 7ms/step - loss: 2.5795 - accuracy: 0.0907 - val_loss: 2.5816 - val_accuracy: 0.104

In [28]:
# evaluate network on test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print("\ntest loss: {}, test accuracy: {}".format(test_loss, 100*test_acc))

18/18 [==============================] - 1s 6ms/step - loss: 0.4145 - accuracy: 0.9028

test loss: 0.41450533270835876, test accuracy: 90.27777910232544


In [30]:
model.save('model.h5')

### **Prediction**

In [40]:
MAX_LEN = 300
def signal2mfcc(file_path, max_len=MAX_LEN):
    signal, sample_rate = lr.load(file_path, mono=True, sr=None)
    mfcc = lr.feature.mfcc(signal, sample_rate)

    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [52]:
def prediction(audio_path):
    audio_file = glob(audio_path)
    MFCCs = signal2mfcc(audio_file[0])
    MFCCs = MFCCs.reshape(1,20,300,1)
    prediction = model.predict(MFCCs)[0]
    print(prediction)
    category_index = np.where(prediction == max(prediction))[0][0]
    print(category_index)
    result = categories[category_index]
    return result

In [53]:
prediction('/content/dataset/19/40-(19).mp3')

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


1/1 [==============================] - 0s 29ms/step
[5.9255871e-07 8.1343472e-08 3.0726619e-09 8.0672107e-05 1.3127899e-13
 6.3199858e-12 9.3408738e-07 9.0253404e-16 5.7830693e-14 2.6646486e-18
 9.9991775e-01 2.1652857e-11]
10


'مشتقات'